 ## Парсинг Часть 2

In [ ]:
# Загружаем библиотеки
import numpy as np # linear algebra
import pandas as pd
import os
import requests  
from bs4 import BeautifulSoup
import time
import json
import re

In [10]:
data = pd.read_csv('links.csv',header=None)
all_links = data[0].tolist()

In [24]:
# Создаем функцию для парсинга даных
def collecting_features(all_links, step, max_step):
    
    # Создаем пустой датафрейм для последующих записей
    df = pd.DataFrame()
        

    count = 0
    for link in all_links[step:max_step]: 
      
        # Создаем пустой датафрейм для цикла
        cycle_df = pd.DataFrame()
        
        # Загружаем страницу по ссылке
        try:
            res = requests.get(link)
            page = BeautifulSoup(res.content.decode('utf-8'), 'html.parser')
        except:
            print(f'Error get link={link}, {res.status_code}')
        
        # Загружаем поочередно страницу с блоком JSON
        try:
            json_data = json.loads(page.find('script', 
                                     type="application/ld+json").string)
        except:
            json_data = None
    
        #  Загружаем блок, где находится ссылка на модель
        try:
            url_block = page.find('a', class_='Link SpoilerLink CardCatalogLink SpoilerLink_type_default')
            brand_model = url_block.get('href')
            model_response = requests.get(brand_model)
            model_page = BeautifulSoup(
                model_response.content.decode('utf-8'), 'html.parser')
        except:
            print(f'Error get link={link}, {res.status_code}')
    
        # Загружаем часть JSON по ссылке модели
        try:
            json_data_model= json.loads(model_page.find('script', 
                            type="application/json", 
                            id='initial-state').string)
        except:
            json_data_model = None    
        
        # Находим перечень опций данной модели по ссылке
        try:
            options_dict = json_data_model['state']['compare']['selected'][0]['options']
            options_list = [key for key in list(options_dict.keys()) if options_dict[key] == 1]
        except:
            options_list = []
        
        # Выводим блок оснащения
        try:
            json_data_equipment = json.loads(
                page.find('script', type="application/json", id='initial-state').string)
        except:
            json_data_equipment = None    
        
    # Собираем необходимые признаки согласно перечню в тестовой выборке
        try:
            bodyType = json_data['bodyType']
        except:
            bodyType = None
        try:
            brand = json_data['brand']
        except:
            brand = None    
        try:
            color = json_data['color']
        except:
            color = None    
        try:
            complectation_dict = options_list
        except:
            complectation_dict = None    
        try:
            description = json_data['description']
        except:
            description = None    
        try:
            engineDisplacement = json_data['vehicleEngine']['engineDisplacement']
        except:
            engineDisplacement = None    
        try:
            enginePower = json_data['vehicleEngine']['enginePower']
        except:
            enginePower = None    
        try:    
            equipment_dict = json_data_equipment['card']['vehicle_info']['equipment']
        except:
            equipment_dict = None    
        try:
            fuelType = json_data['fuelType']
        except:
            fuelType = None    
        try:
            mileage = page.find('li', 
                        class_='CardInfoRow CardInfoRow_kmAge').find_all(
            'span')[-1].text.replace(u'\xa0', u' ')
        except:
            mileage = None    
        try:
            modelDate = json_data['modelDate']
        except:
            modelDate = None    
        try:
            model_info = json_data_equipment['card']['vehicle_info']['model_info']
        except:
            model_info = None    
        try:
            model_name = page.find_all(
                'div', class_='InfoPopup InfoPopup_theme_plain InfoPopup_withChildren BreadcrumbsPopup')[1].text
        except:
            model_name = None    
        try:
            name = json_data['name']
        except:
            name = None
        try:
            numberOfDoors = json_data['numberOfDoors']
        except:
            numberOfDoors = None
        try:
            parsing_unixtime = int(time.time())
        except:
            parsing_unixtime = None    
        try:
            productionDate = json_data['productionDate']
        except:
            productionDate = None    
        try:
            sell_id = page.find(
            'div', class_='CardHead__infoItem CardHead__id').text[2:]
        except:
            sell_id = None    
        try:
            super_gen = json.loads(
                page.find('div', id="sale-data-attributes").get('data-bem'))
        except:
            super_gen = None
        try:
            vehicleConfiguration = json_data['vehicleConfiguration']
        except:
            vehicleConfiguration = None    
        try:
            vehicleTransmission = json_data['vehicleTransmission']
        except:
            vehicleTransmission = None    
        try:
            vendor = json_data_equipment['card']['vehicle_info']['vendor']
        except:
            vendor = None    
        try:
            Владельцы = page.find('li', class_='CardInfoRow CardInfoRow_ownersCount').find_all(
                'span')[-1].text.replace(u'\xa0', u' ')
        except:
            Владельцы = None    
        
        try:
            Владение = page.find(
                'li', class_='CardInfoRow CardInfoRow_owningTime').find_all('span')[-1].text
        except:
            Владение = None    
        try:
            ПТС = page.find('li', class_='CardInfoRow CardInfoRow_pts').find_all(
                'span')[-1].text
        except:
            ПТС = None    
        try:
            Привод = page.find('li', class_='CardInfoRow CardInfoRow_drive').find_all(
                'span')[-1].text
        except:
            Привод = None    
        try:
            Руль = page.find('li', class_='CardInfoRow CardInfoRow_wheel').find_all(
                'span')[-1].text
        except:
            Руль = None
        try:
            Состояние = page.find(
                'li', class_='CardInfoRow CardInfoRow_state').find_all('span')[-1].text
        except:
            Состояние = None
        try:
            Таможня = page.find(
                'li', class_='CardInfoRow CardInfoRow_customs').find_all('span')[-1].text
        except:
            Таможня = None
        
        # Запишем дату объявления    
        try:
            advert_date = page.find(
                'div', class_='CardHead__infoItem CardHead__creationDate').text
        except:
            advert_date = None
        
        # Цена предложения. Отсутствующий параметр в тесте - нужно предсказать
        try:
            price = json_data['offers']['price']
        except:
            price = None
        
        
        # Создадим строку из словаря признаков и их значений:
        row = {'bodyType': bodyType,
               'brand': brand,
               'color': color,
               'complectation_dict': complectation_dict, 
               'description': description,
               'engineDisplacement': engineDisplacement, 
               'enginePower': enginePower,
               'equipment_dict': equipment_dict, 
               'fuelType': fuelType, 
               'mileage': mileage, 
               'modelDate': modelDate,
               'model_info': model_info,
               'model_name': model_name,
               'name': name, 
               'numberOfDoors': numberOfDoors, 
               'parsing_unixtime': parsing_unixtime,  
               'productionDate': productionDate,
               'sell_id': sell_id, 
               'super_gen': super_gen,
               'vehicleConfiguration': vehicleConfiguration,
               'vehicleTransmission': vehicleTransmission,
               'vendor': vendor,
               'Владельцы': Владельцы,
               'Владение': Владение,
               'ПТС': ПТС, 
               'Привод': Привод, 
               'Руль': Руль,
               'Состояние': Состояние, 
               'Таможня': Таможня,  
               'advert_date': advert_date,
               'price': price
              }
        
        df = pd.concat([df, pd.DataFrame([row])])

        # увеличиваем смещение на количество строк выгрузки
        time.sleep(0.05)
        step += 1
        count += 1
        if step % 100 == 0:
            time.sleep(2)
            print(f'Downloaded from line: {step} counts at the current cycle {count}')
        else:
            continue
    return df

In [35]:
basic_df = pd.DataFrame()

In [31]:
data_1 = collecting_features(all_links, 0, 1000)
data_1.to_csv('data_1.csv', index=False)

Downloaded from line: 100 counts at the current cycle 100
Downloaded from line: 200 counts at the current cycle 200
Downloaded from line: 300 counts at the current cycle 300
Downloaded from line: 400 counts at the current cycle 400
Downloaded from line: 500 counts at the current cycle 500
Downloaded from line: 600 counts at the current cycle 600
Downloaded from line: 700 counts at the current cycle 700
Downloaded from line: 800 counts at the current cycle 800
Downloaded from line: 900 counts at the current cycle 900
Downloaded from line: 1000 counts at the current cycle 1000


In [36]:
basic_df = pd.concat([basic_df, data_1])

In [17]:
data_2 = collecting_features(all_links, 2000, 3000)

Downloaded from line: 2100 counts at the current cycle 100
Downloaded from line: 2200 counts at the current cycle 200
Downloaded from line: 2300 counts at the current cycle 300
Downloaded from line: 2400 counts at the current cycle 400
Downloaded from line: 2500 counts at the current cycle 500
Downloaded from line: 2600 counts at the current cycle 600
Downloaded from line: 2700 counts at the current cycle 700
Downloaded from line: 2800 counts at the current cycle 800
Downloaded from line: 2900 counts at the current cycle 900
Downloaded from line: 3000 counts at the current cycle 1000


In [20]:
data_2.to_csv('data_2.csv', index=False)

In [37]:
basic_df = pd.concat([basic_df, data_2])

In [21]:
data_3 = collecting_features(all_links, 3000, 4000)

Error get link=https://auto.ru/cars/used/sale/mitsubishi/galant/1105143145-c892091d/, 200
Downloaded from line: 3100 counts at the current cycle 100
Downloaded from line: 3200 counts at the current cycle 200
Error get link=https://auto.ru/cars/used/sale/mitsubishi/carisma/1103043432-7fe216bb/, 200
Downloaded from line: 3300 counts at the current cycle 300
Downloaded from line: 3400 counts at the current cycle 400
Downloaded from line: 3500 counts at the current cycle 500
Error get link=https://auto.ru/cars/used/sale/mitsubishi/carisma/1101461701-bc78c558/, 200
Downloaded from line: 3600 counts at the current cycle 600
Downloaded from line: 3700 counts at the current cycle 700
Error get link=https://auto.ru/cars/used/sale/volkswagen/touareg/1105328044-f661f279/, 200
Downloaded from line: 3800 counts at the current cycle 800
Downloaded from line: 3900 counts at the current cycle 900
Error get link=https://auto.ru/cars/used/sale/volkswagen/tiguan/1105326602-0e618ed9/, 200
Downloaded from 

In [22]:
data_3.to_csv('data_3.csv', index=False)

In [38]:
basic_df = pd.concat([basic_df, data_3])

In [25]:
data_4 = collecting_features(all_links, 4000, 8000)
data_4.to_csv('data_4.csv', index=False)

Downloaded from line: 4100 counts at the current cycle 100
Downloaded from line: 4200 counts at the current cycle 200
Downloaded from line: 4300 counts at the current cycle 300
Downloaded from line: 4400 counts at the current cycle 400
Downloaded from line: 4500 counts at the current cycle 500
Downloaded from line: 4600 counts at the current cycle 600
Downloaded from line: 4700 counts at the current cycle 700
Downloaded from line: 4800 counts at the current cycle 800
Downloaded from line: 4900 counts at the current cycle 900
Downloaded from line: 5000 counts at the current cycle 1000
Downloaded from line: 5100 counts at the current cycle 1100
Downloaded from line: 5200 counts at the current cycle 1200
Downloaded from line: 5300 counts at the current cycle 1300
Downloaded from line: 5400 counts at the current cycle 1400
Error get link=https://auto.ru/cars/used/sale/volkswagen/polo/1105237518-c1b416fc/, 200
Downloaded from line: 5500 counts at the current cycle 1500
Downloaded from line:

In [39]:
basic_df = pd.concat([basic_df, data_4])

In [27]:
data_5 = collecting_features(all_links, 8000, 13000)
data_5.to_csv('data_5.csv', index=False)

Downloaded from line: 8100 counts at the current cycle 100
Downloaded from line: 8200 counts at the current cycle 200
Downloaded from line: 8300 counts at the current cycle 300
Downloaded from line: 8400 counts at the current cycle 400
Downloaded from line: 8500 counts at the current cycle 500
Downloaded from line: 8600 counts at the current cycle 600
Downloaded from line: 8700 counts at the current cycle 700
Error get link=https://auto.ru/cars/used/sale/skoda/rapid/1105053165-27e018d5/, 200
Downloaded from line: 8800 counts at the current cycle 800
Downloaded from line: 8900 counts at the current cycle 900
Error get link=https://auto.ru/cars/used/sale/skoda/octavia/1105268215-0294cb64/, 503
Downloaded from line: 9000 counts at the current cycle 1000
Downloaded from line: 9100 counts at the current cycle 1100
Downloaded from line: 9200 counts at the current cycle 1200
Downloaded from line: 9300 counts at the current cycle 1300
Downloaded from line: 9400 counts at the current cycle 1400

In [40]:
basic_df = pd.concat([basic_df, data_5])

In [28]:
data_6 = collecting_features(all_links, 13000, 20000)
data_6.to_csv('data_6.csv', index=False)

Downloaded from line: 13100 counts at the current cycle 100
Downloaded from line: 13200 counts at the current cycle 200
Downloaded from line: 13300 counts at the current cycle 300
Downloaded from line: 13400 counts at the current cycle 400
Downloaded from line: 13500 counts at the current cycle 500
Error get link=https://auto.ru/cars/used/sale/bmw/5er/1094795740-1048c0fd/, 599
Downloaded from line: 13600 counts at the current cycle 600
Error get link=https://auto.ru/cars/used/sale/bmw/x5/1105169199-214cf2f1/, 200
Downloaded from line: 13700 counts at the current cycle 700
Downloaded from line: 13800 counts at the current cycle 800
Downloaded from line: 13900 counts at the current cycle 900
Downloaded from line: 14000 counts at the current cycle 1000
Downloaded from line: 14100 counts at the current cycle 1100
Downloaded from line: 14200 counts at the current cycle 1200
Downloaded from line: 14300 counts at the current cycle 1300
Downloaded from line: 14400 counts at the current cycle 1

In [41]:
basic_df = pd.concat([basic_df, data_6])

In [29]:
data_7 = collecting_features(all_links, 20000, 26000)
data_7.to_csv('data_7.csv', index=False)

Downloaded from line: 20100 counts at the current cycle 100
Downloaded from line: 20200 counts at the current cycle 200
Downloaded from line: 20300 counts at the current cycle 300
Downloaded from line: 20400 counts at the current cycle 400
Downloaded from line: 20500 counts at the current cycle 500
Downloaded from line: 20600 counts at the current cycle 600
Downloaded from line: 20700 counts at the current cycle 700
Downloaded from line: 20800 counts at the current cycle 800
Downloaded from line: 20900 counts at the current cycle 900
Downloaded from line: 21000 counts at the current cycle 1000
Downloaded from line: 21100 counts at the current cycle 1100
Downloaded from line: 21200 counts at the current cycle 1200
Downloaded from line: 21300 counts at the current cycle 1300
Downloaded from line: 21400 counts at the current cycle 1400
Downloaded from line: 21500 counts at the current cycle 1500
Downloaded from line: 21600 counts at the current cycle 1600
Error get link=https://auto.ru/ca

In [42]:
basic_df = pd.concat([basic_df, data_7])

In [30]:
data_8 = collecting_features(all_links, 26000, 31334)
data_8.to_csv('data_8.csv', index=False)

Downloaded from line: 26100 counts at the current cycle 100
Downloaded from line: 26200 counts at the current cycle 200
Downloaded from line: 26300 counts at the current cycle 300
Downloaded from line: 26400 counts at the current cycle 400
Downloaded from line: 26500 counts at the current cycle 500
Downloaded from line: 26600 counts at the current cycle 600
Downloaded from line: 26700 counts at the current cycle 700
Downloaded from line: 26800 counts at the current cycle 800
Downloaded from line: 26900 counts at the current cycle 900
Downloaded from line: 27000 counts at the current cycle 1000
Downloaded from line: 27100 counts at the current cycle 1100
Error get link=https://auto.ru/cars/used/sale/toyota/rav_4/1105303558-a2017d6a/, 502
Error get link=https://auto.ru/cars/used/sale/toyota/camry/1105106437-61f27bad/, 502
Downloaded from line: 27200 counts at the current cycle 1200
Error get link=https://auto.ru/cars/used/sale/toyota/land_cruiser/1105272525-095a1f27/, 502
Error get link=

In [43]:
basic_df = pd.concat([basic_df, data_8])

In [32]:
# Запишем линки с выданными ошибками
error_list = ['https://auto.ru/cars/used/sale/volkswagen/polo/1105237518-c1b416fc/',
              'https://auto.ru/cars/used/sale/volkswagen/polo/1105311247-b4d81614/',
              'https://auto.ru/cars/used/sale/volkswagen/passat/1086264362-dc44fd02/',
              'https://auto.ru/cars/used/sale/volkswagen/caravelle/1105171518-cb617768/',
              'https://auto.ru/cars/used/sale/skoda/octavia/1105328813-f7e3073b/',
              'https://auto.ru/cars/used/sale/skoda/rapid/1105053165-27e018d5/',
              'https://auto.ru/cars/used/sale/skoda/octavia/1105268215-0294cb64/',
              'https://auto.ru/cars/used/sale/skoda/fabia/1096464370-d453307d/',
              'https://auto.ru/cars/used/sale/bmw/x6/1105021835-5cdfa18a/',
              'https://auto.ru/cars/used/sale/bmw/x3/1104240832-2f13885b/',
              'https://auto.ru/cars/used/sale/bmw/3er/1105298818-651a9783',
              'https://auto.ru/cars/used/sale/bmw/x5_m/1105298338-fec8f08d/',
              'https//auto.ru/cars/used/sale/bmw/x3/1105234930-5e948952/',
              'https://auto.ru/cars/used/sale/bmw/3er/1101150000-80a3f533/',
              'https://auto.ru/cars/used/sale/bmw/5er/1105226137-28a78a2e/',
              'https://auto.ru/cars/used/sale/bmw/3er/1105184797-f221fb3e/',
              'https://auto.ru/cars/used/sale/bmw/5er/1094795740-1048c0fd/',
              'https://auto.ru/cars/used/sale/bmw/x5/1105169199-214cf2f1/',
              'https://auto.ru/cars/used/sale/audi/q7/1104902328-11078536/',
              'https://auto.ru/cars/used/sale/nissan/murano/1105044168-f185a884/',
              'https://auto.ru/cars/used/sale/mercedes/m_klasse/1104772436-fd751777/',
              'https://auto.ru/cars/used/sale/mercedes/v_klasse/1105315766-9bc676d5/',
              'https://auto.ru/cars/used/sale/mercedes/gl_klasse/1105313076-0b342b3c/',
              'https://auto.ru/cars/used/sale/mercedes/s_klasse/1105146953-cfc50ea2/',
              'https://auto.ru/cars/used/sale/toyota/rav_4/1105303558-a2017d6a/',
              'https://auto.ru/cars/used/sale/toyota/camry/1105106437-61f27bad/',
              'https://auto.ru/cars/used/sale/toyota/land_cruiser/1105272525-095a1f27/',
              'https://auto.ru/cars/used/sale/toyota/land_cruiser_prado/1105272978-e3f66587/',
              'https://auto.ru/cars/used/sale/toyota/tundra/1105197954-828b10ad/',
              'https://auto.ru/cars/used/sale/toyota/camry/1105160682-7900d811/',
              'https://auto.ru/cars/used/sale/toyota/rav_4/1105187520-e955e843/',
              'https://auto.ru/cars/used/sale/toyota/land_cruiser_prado/1105255778-1feb234f/',
              'https://auto.ru/cars/used/sale/toyota/alphard/1105162341-b81f3d7a/',
              'https://auto.ru/cars/used/sale/toyota/rav_4/1105150265-2c766158/',
              'https://auto.ru/cars/used/sale/lexus/rx/1105263984-a2cea687/',
              'https://auto.ru/cars/used/sale/lexus/nx/1105255610-f4a5d5a9/'
]

In [ ]:
len(error_list)

In [34]:
data_9 = collecting_features(error_list, 0, len(error_list))
data_9.to_csv('data_9.csv', index=False)

Error get link=https://auto.ru/cars/used/sale/volkswagen/polo/1105237518-c1b416fc/, 200
Error get link=https://auto.ru/cars/used/sale/volkswagen/polo/1105311247-b4d81614/, 200
Error get link=https://auto.ru/cars/used/sale/volkswagen/passat/1086264362-dc44fd02/, 200
Error get link=https://auto.ru/cars/used/sale/skoda/octavia/1105328813-f7e3073b/, 200
Error get link=https://auto.ru/cars/used/sale/skoda/rapid/1105053165-27e018d5/, 200
Error get link=https://auto.ru/cars/used/sale/bmw/x5_m/1105298338-fec8f08d/, 200
Error get link=https//auto.ru/cars/used/sale/bmw/x3/1105234930-5e948952/, 200
Error get link=https//auto.ru/cars/used/sale/bmw/x3/1105234930-5e948952/, 200
Error get link=https://auto.ru/cars/used/sale/bmw/x5/1105169199-214cf2f1/, 200
Error get link=https://auto.ru/cars/used/sale/audi/q7/1104902328-11078536/, 200
Error get link=https://auto.ru/cars/used/sale/nissan/murano/1105044168-f185a884/, 200
Error get link=https://auto.ru/cars/used/sale/mercedes/v_klasse/1105315766-9bc676d

In [44]:
basic_df = pd.concat([basic_df, data_9])

In [57]:
basic_df.to_csv('basic_df.csv')

## Очистка данных 

In [449]:
new_df = pd.read_csv('basic_df.csv')

In [450]:
new_df = new_df.drop('Unnamed: 0', axis=1)

In [451]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30369 entries, 0 to 30368
Data columns (total 31 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bodyType              30314 non-null  object 
 1   brand                 30314 non-null  object 
 2   color                 30314 non-null  object 
 3   complectation_dict    30369 non-null  object 
 4   description           30314 non-null  object 
 5   engineDisplacement    30314 non-null  object 
 6   enginePower           30314 non-null  object 
 7   equipment_dict        30314 non-null  object 
 8   fuelType              30314 non-null  object 
 9   mileage               30314 non-null  object 
 10  modelDate             30314 non-null  float64
 11  model_info            30314 non-null  object 
 12  model_name            30314 non-null  object 
 13  name                  30347 non-null  object 
 14  numberOfDoors         30314 non-null  float64
 15  parsing_unixtime   

In [452]:
# Посмотрим на количество пропусков в sell_id
new_df['sell_id'].isna().value_counts()

False    30314
True        55
Name: sell_id, dtype: int64

In [453]:
new_df = new_df.dropna(subset=['sell_id'], axis=0)

In [454]:
# Проверим, что удалились пропуски
new_df['sell_id'].isna().value_counts()

False    30314
Name: sell_id, dtype: int64

In [455]:
# Удалим строки без указанной цены
new_df = new_df.dropna(subset=['price'], axis=0)

In [456]:
# Проверим
new_df.price.isna().value_counts()

False    28165
Name: price, dtype: int64

In [457]:
# Удалим дупликаты
new_df = new_df.drop_duplicates(subset=['sell_id'])

In [458]:
new_df.shape

(25031, 31)

In [459]:
new_df.price.isna().value_counts()

False    25031
Name: price, dtype: int64

В тестовой выборке в признаке пробег указаны числовые данные. Необходимо преобразовать признак в обучающей выборке

In [502]:
test = new_df.copy()

In [503]:
# В признаке пробега присутствуют пропуски
test.mileage.isna().value_counts()

False    25031
Name: mileage, dtype: int64

In [504]:
# Посмотрим на распределение
test.mileage.value_counts()

200 000 км    277
300 000 км    243
250 000 км    242
180 000 км    221
150 000 км    199
             ... 
190 032 км      1
128 440 км      1
147 679 км      1
35 670 км       1
156 891 км      1
Name: mileage, Length: 10239, dtype: int64

In [505]:
# Заполним наиболее часто встречающимся значением
test.mileage.fillna('200 000 км', inplace=True)

In [506]:
test.mileage.isna().value_counts()

False    25031
Name: mileage, dtype: int64

In [507]:
# Избавляемся от км
test.mileage = test.mileage.apply(lambda x: x[:-3])

In [508]:
test.mileage.value_counts()

200 000    277
300 000    243
250 000    242
180 000    221
150 000    199
          ... 
95 376       1
900          1
131 838      1
101 164      1
152 840      1
Name: mileage, Length: 10239, dtype: int64

In [509]:
# Преобразуем в целые числа
test['mileage'] = test['mileage'].apply(lambda x: x.split(' ') if type(x) == str  else x)
test['mileage'] = test['mileage'].apply(lambda x: x[0]+x[1] if len(x) > 1 else x)
test['mileage'] = test['mileage'].apply(lambda x: x.strip('[').strip(']') if type(x) == str else x)
test['mileage'] = test['mileage'].apply(lambda x: int(x) if type(x) == str else int(x[0]))

In [510]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25031 entries, 0 to 30365
Data columns (total 31 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bodyType              25031 non-null  object 
 1   brand                 25031 non-null  object 
 2   color                 25031 non-null  object 
 3   complectation_dict    25031 non-null  object 
 4   description           25031 non-null  object 
 5   engineDisplacement    25031 non-null  object 
 6   enginePower           25031 non-null  object 
 7   equipment_dict        25031 non-null  object 
 8   fuelType              25031 non-null  object 
 9   mileage               25031 non-null  int64  
 10  modelDate             25031 non-null  float64
 11  model_info            25031 non-null  object 
 12  model_name            25031 non-null  object 
 13  name                  25031 non-null  object 
 14  numberOfDoors         25031 non-null  float64
 15  parsing_unixtime   

После парсинга ключевой задачей являлось собрать датасе, в котором нет пропусков в цене и sell_id. Важно соответствие типов данных. Как поступать с пропусками в других колонках определимся далее

In [511]:
test.to_csv('auto_df.csv')